In [26]:
import pandas as pd

In [5]:
df = pd.read_csv('../dados_titanic/train.csv')
df_kaggle = pd.read_csv('../dados_titanic/test.csv')

In [6]:
df = df.append(df_kaggle, sort=False)

In [7]:
for col in df.columns:
    if df[col].dtype == 'object':
        df[col] = df[col].astype('category')
        df[col] = df[col].cat.codes

In [8]:
df.fillna(-1, inplace=True)

In [9]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,155,1,22.0,1,0,720,7.2500,-1,2
1,2,1.0,1,286,0,38.0,1,0,816,71.2833,106,0
2,3,1.0,3,523,0,26.0,0,0,914,7.9250,-1,2
3,4,1.0,1,422,0,35.0,1,0,65,53.1000,70,2
4,5,0.0,3,22,1,35.0,0,0,649,8.0500,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,-1.0,3,1126,1,-1.0,0,0,711,8.0500,-1,2
414,1306,-1.0,1,891,0,39.0,0,0,834,108.9000,64,0
415,1307,-1.0,3,1046,1,38.5,0,0,872,7.2500,-1,2
416,1308,-1.0,3,1227,1,-1.0,0,0,579,8.0500,-1,2


In [10]:
df_kaggle = df.iloc[891:].copy()
df = df.iloc[:891].copy()

In [27]:
df

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0.0,3,155,1,22.0,1,0,720,7.2500,-1,2
1,2,1.0,1,286,0,38.0,1,0,816,71.2833,106,0
2,3,1.0,3,523,0,26.0,0,0,914,7.9250,-1,2
3,4,1.0,1,422,0,35.0,1,0,65,53.1000,70,2
4,5,0.0,3,22,1,35.0,0,0,649,8.0500,-1,2
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0.0,2,811,1,27.0,0,0,130,13.0000,-1,2
887,888,1.0,1,464,0,19.0,0,0,18,30.0000,40,2
888,889,0.0,3,607,0,-1.0,1,2,923,23.4500,-1,2
889,890,1.0,1,113,1,26.0,0,0,11,30.0000,77,0


In [12]:
from sklearn.model_selection import train_test_split

In [13]:
train, teste = train_test_split(df, test_size=0.2, random_state=5)

In [14]:
remove = ['Survived','PassengerId','Name']
features = [col for col in df.columns if col not in remove]

In [15]:
features

['Pclass',
 'Sex',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [48]:
passager_dict = {'Pclass': 2,
 'Sex': 0,
 'Age': 24,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': 522,
 'Fare': 7.5500,
 'Cabin': 69,
 'Embarked': 0}

passager_dict

{'Pclass': 2,
 'Sex': 0,
 'Age': 24,
 'SibSp': 0,
 'Parch': 0,
 'Ticket': 522,
 'Fare': 7.55,
 'Cabin': 69,
 'Embarked': 0}

In [61]:
passager_df = pd.DataFrame(passager_dict, index=[0])

In [62]:
passager_df

,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,2,0,24,0,0,522,7.55,69,0


In [16]:
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

In [19]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [162]:
clf = LinearSVC(C=1, random_state=5, penalty='l1', loss='squared_hinge', dual=False)
clf.fit(train[features], train['Survived'])

LinearSVC(C=1, dual=False, penalty='l1', random_state=5)

In [165]:
pred = clf.predict(teste[features])
mt_cf = confusion_matrix(teste['Survived'], pred)
print(f"""
{accuracy_score(teste['Survived'], pred)*100}
{mt_cf}""")


79.88826815642457
[[98 13]
 [23 45]]


In [158]:
clf2 = SGDClassifier(loss='perceptron', penalty='elasticnet', alpha=0.09009, max_iter=50, tol=0.03, random_state=5, )
clf2.fit(train[features], train['Survived'])

SGDClassifier(alpha=0.09009, loss='perceptron', max_iter=50,
              penalty='elasticnet', random_state=5, tol=0.03)

In [160]:
pred = clf2.predict(teste[features])
mt_cf = confusion_matrix(teste['Survived'], pred)
print(f"""
{accuracy_score(teste['Survived'], pred)*100}
{mt_cf}""")


82.12290502793296
[[94 17]
 [15 53]]


In [92]:
clf3 = RandomForestClassifier(n_jobs=-1, random_state=48, class_weight='balanced_subsample')
clf3.fit(train[features], train['Survived'])

RandomForestClassifier(class_weight='balanced_subsample', n_jobs=-1,
                       random_state=48)

In [167]:
pred = clf3.predict(teste[features])
mt_cf = confusion_matrix(teste['Survived'], pred)
print(f"""
{accuracy_score(teste['Survived'], pred)*100}
{mt_cf}""")


83.79888268156425
[[102   9]
 [ 20  48]]


In [154]:
clf4 = MLPClassifier(activation='logistic', solver='adam', alpha=0.03, max_iter=500, learning_rate_init=0.001,  random_state=48, hidden_layer_sizes=(9,18,13))
clf4.fit(train[features], train['Survived'])

MLPClassifier(activation='logistic', alpha=0.03, hidden_layer_sizes=(9, 18, 13),
              max_iter=500, random_state=48)

In [157]:
pred = clf4.predict(teste[features])
mt_cf = confusion_matrix(teste['Survived'], pred)
print(f"""
{accuracy_score(teste['Survived'], pred)*100}
{mt_cf}""")


83.79888268156425
[[97 14]
 [15 53]]


In [168]:
result = clf3.predict(df_kaggle[features])

In [68]:
comparation = pd.read_csv('../dados_titanic/possivel saida.csv')
comparation

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [169]:
count = 0
for my_r, comp in zip(result, comparation['Survived']):
    if my_r != comp:
        count += 1

In [170]:
count

64